In [1]:
import numpy as np
import codecs
class FastVector:
    """
    Minimal wrapper for fastvector embeddings.
    ```
    Usage:
        $ model = FastVector(vector_file='/path/to/wiki.en.vec')
        $ 'apple' in model
        > TRUE
        $ model['apple'].shape
        > (300,)
    ```
    """

    def __init__(self, vector_file='', transform=None):
        """Read in word vectors in fasttext format"""
        self.word2id = {}

        # Captures word order, for export() and translate methods
        self.id2word = []

        print('reading word vectors from %s' % vector_file)
        with open(vector_file, 'r') as f:
	    print ('1') 
            (self.n_words, self.n_dim) = \
            (int(x) for x in f.readline().rstrip('\n').split(' '))
            self.embed = np.zeros((self.n_words, self.n_dim))
            for i, line in enumerate(f):
                elems = line.rstrip('\n').split(' ')
                self.word2id[elems[0]] = i
                #print (elems[0])
                ini=np.random.rand(300)
                self.embed[i] = ini[:] #elems[1:self.n_dim+1]
                self.id2word.append(elems[0])
        
        # Used in translate_inverted_softmax()
        self.softmax_denominators = None
        
        if transform is not None:
            print('Applying transformation to embedding')
            self.apply_transform(transform)
    
    def apply_cop(self, matrix,i):
        self.embed[i]=matrix[:]
    
    def export(self, outpath):
        """
        Transforming a large matrix of WordVectors is expensive. 
        This method lets you write the transformed matrix back to a file for future use
        :param The path to the output file to be written 
        """
        fout = open(outpath, "w")

        # Header takes the guesswork out of loading by recording how many lines, vector dims
        fout.write(str(self.n_words) + " " + str(self.n_dim) + "\n")
        for token in self.id2word:
            vector_components = ["%.6f" % number for number in self[token]]
            vector_as_string = " ".join(vector_components)

            out_line = token + " " + vector_as_string + "\n"
            fout.write(out_line)

        fout.close()
    
    
    @classmethod
    def cosine_similarity(cls, vec_a, vec_b):
        """Compute cosine similarity between vec_a and vec_b"""
        return np.dot(vec_a, vec_b) / \
            (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))

    def __contains__(self, key):
        return key in self.word2id

    def __getitem__(self, key):
        return self.embed[self.word2id[key]]

In [2]:
import numpy as np
from random import randint

In [3]:
def make_training_matrices(source_dictionary, target_dictionary, bilingual_dictionary):
    """
    Source and target dictionaries are the FastVector objects of
    source/target languages. bilingual_dictionary is a list of 
    translation pair tuples [(source_word, target_word), ...].
    """
    source_matrix = []
    target_matrix = []
    ti=[]
    count=0
    for (source, target) in bilingual_dictionary:
        #print source,target
        if source in source_dictionary and target in target_dictionary:
            #print source, target
            count=count+1
            #print source, target
            #print source+1
            x=randint(0,len(source_dictionary.word2id))

            #print x
            source1= source_dictionary.id2word[x]
            print source, source1,target
            ti.append(target_dictionary.word2id[target])
            source_matrix.append(source_dictionary[source1])
            target_matrix.append(target_dictionary[target])
        
    # return training matrices
    print count
    return np.array(source_matrix), np.array(target_matrix), np.array(ti)

In [4]:
en_dictionary = FastVector(vector_file='/home/apatra/fastText/fastText_multilingual-master/eng.vec')
mi_dictionary = FastVector(vector_file='/home/apatra/fastText/fastText_multilingual-master/model.vec')

en_vector = en_dictionary["one"]
mi_vector = mi_dictionary["newt"]
print(FastVector.cosine_similarity(en_vector, mi_vector))

reading word vectors from /home/apatra/fastText/fastText_multilingual-master/eng.vec
1


MemoryError: 

In [9]:
mi_words = set(mi_dictionary.word2id.keys())
en_words = set(en_dictionary.word2id.keys())

In [10]:
import codecs
bilingual_dictionary=[]
with codecs.open('/home/apatra/fastText/fastText_multilingual-master/eng-mic','r','utf-8') as f:
    for line in f:
        eng, mic=line.split(', ')
        #print eng
        eng=eng.strip('\"')
        #print eng
        mic=mic.strip('\"')
        mic=mic.replace('\n','')
        mic=mic.replace('"','')
        #print eng, mic
        bilingual_dictionary.append((eng,mic))
#print bilingual_dictionary

In [11]:
# form the training matrices
#from copy import deepcopy
source_matrix, target_matrix ,ti= make_training_matrices(
    en_dictionary, mi_dictionary, bilingual_dictionary)
#print len(source_matrix), len(target_matrix)
# learn and apply the transformation
#print ti, len(ti)
#target_matrix=deepcopy(source_matrix)
#print source_matrix #[60][9], target_matrix[60][9]
#transform = learn_transformation(source_matrix, target_matrix)
#print type(transform)
#print transform[299]
#en_dictionary.apply_transform(transform)

choose do.Then megnatl
I I-a nin
aboard temovate teppit
aboriginal downscaled Lnu
abstruse comptons temig
adequate AM-7 tepiet
adequate Aloe-Vera tepiaq
again N1C app
alive max. mimajit
allow treach ignmuatl
almost .vsd suel
also Tarbet elg
also Gta5 jel
always OurPets apjiw
and Świata jel
and servatory aq
another ALAP igtig
arrive you-but pegising
aside Akin gmetug
asleep RRSO nepat
at Grabez eteg
attached cable-access naspit
attached fisico nasteg
authority Lyonel alsusuti
aware Genesius gejiatl
ay Marovo amuj
aye anti-alpha amuj
battle 10-17-2010 matntimg
battle 15-piece matnaggewaqan
bear postsynaptically muin
beaver Sq.Ft. kopit
because 1930s-style muta
before 12.5.2017 tmg
before dandy gesgmnaq
beside Bird-B-Gone gmetug
beside 28.55 anapiw
blacksmith seana klaptan
blaze Kaupanger gnugwaqan
bleed Ceej maltewiaq
bless Tiazac elapatoq
boss IslandDay alsusit
boss 1.715 assusit
both self-rightous gitg
build Settsu eltoq
but Mn gatu
Canada Jaymie Ganata
canoe years--or kwitn
caribou vi

In [12]:
from collections import defaultdict

def list_duplicates(seq, x):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    return (locs for key,locs in tally.items() 
            if key==x)

'''
ind=6
p=list_duplicates(ti,ti[ind])
#for l in p:
 #   print l
j=np.zeros(300)
print source_matrix[ind]
print source_matrix[191]
for l in p:
    for x in l:
        j+=source_matrix[x]
            
    target_matrix[ind]=j[:]/len(l)
print target_matrix[ind]
'''

'\nind=6\np=list_duplicates(ti,ti[ind])\n#for l in p:\n #   print l\nj=np.zeros(300)\nprint source_matrix[ind]\nprint source_matrix[191]\nfor l in p:\n    for x in l:\n        j+=source_matrix[x]\n            \n    target_matrix[ind]=j[:]/len(l)\nprint target_matrix[ind]\n'

In [13]:
import copy
count_no=0
j=np.zeros(300)
for r in range(0,len(ti)):
    #print source_matrix[r], target_matrix[r]
    #print len(source_matrix[r]),len(target_matrix[r])
    print ti[r]
    p=list_duplicates(ti,ti[r])
    #print p
    j=np.zeros(300)
    for l in p:
        print l
        for x in l:
            j+=source_matrix[x]
            
        target_matrix[r]=j[:]/len(l)
    count_no+=1
    #target_matrix[r]=source_matrix[r][:]
    mi_dictionary.apply_cop(target_matrix[r],ti[r])
print count_no

17377
[0, 91, 206]
1474
[1]
16835
[2]
7099
[3, 147]
16371
[4, 57, 181]
9254
[5, 74, 234]
578
[6, 75, 233]
52
[7, 189]
13258
[8]
17285
[9, 171]
483
[10]
648
[11, 254]
19
[12, 14]
759
[13]
19
[12, 14]
1
[15]
629
[16, 164]
1137
[17]
3340
[18, 34]
3858
[19, 282]
683
[20, 127, 174, 215]
1350
[21, 135]
5357
[22, 73]
2411
[23, 207]
5113
[24]
945
[25, 26, 274, 275, 276]
945
[25, 26, 274, 275, 276]
17133
[27, 263]
16938
[28, 81, 231]
2232
[29]
1291
[30]
38
[31]
1185
[32, 85]
316
[33]
3340
[18, 34]
12258
[35]
11959
[36]
14274
[37, 132]
18793
[38]
3964
[39]
2260
[40, 61, 165, 235]
3637
[41, 60, 166, 236]
2036
[42]
1365
[43, 67, 130]
43
[44]
16956
[45]
20368
[46]
6636
[47, 186]
14312
[48, 109, 204, 227, 257]
17140
[49]
7862
[50, 119, 219]
20729
[51]
20551
[52]
1198
[53, 255]
17138
[54]
14220
[55]
13218
[56, 265]
16371
[4, 57, 181]
5137
[58]
14680
[59, 126, 279]
3637
[41, 60, 166, 236]
2260
[40, 61, 165, 235]
16504
[62, 152]
889
[63, 136, 154, 190, 230]
17280
[64]
1776
[65]
7793
[66]
1365
[43, 67, 

In [14]:
mi_dictionary.export('/home/apatra/fastText/fastText_multilingual-master/micmaq5.vec')

In [ ]:
for i in range (0,len(bilingual_dictionary)):
    print bilingual_dictionary[i]

In [ ]:
from random import randint
x=randint(0,len(en_dictionary.word2id))

print x
print en_dictionary.id2word[x]

In [ ]:
print len(en_dictionary.word2id)

In [2]:
import numpy as np
x=np.random.rand(300)
print x

[0.48037991 0.79621893 0.10069728 0.0968313  0.65611778 0.93605489
 0.80080808 0.85866492 0.8851745  0.5600404  0.85136042 0.0253707
 0.96636468 0.82165748 0.33783592 0.25417361 0.12928246 0.693545
 0.07118339 0.89925375 0.09592868 0.63610351 0.85894757 0.92270126
 0.3509721  0.77314531 0.21747356 0.79724538 0.72549369 0.75389477
 0.05140413 0.53005763 0.01587389 0.49694558 0.25731754 0.84034813
 0.54679512 0.32293218 0.60883597 0.73285294 0.50771891 0.58769128
 0.76542808 0.09679181 0.10155699 0.04143743 0.74401417 0.19379413
 0.51043834 0.04775789 0.6332428  0.26262762 0.88869964 0.08717218
 0.15569276 0.7792916  0.0655121  0.55546958 0.41862344 0.96354461
 0.7146249  0.51080211 0.03004831 0.05236235 0.19358396 0.73465263
 0.76831993 0.63603409 0.87747971 0.05998253 0.3443728  0.14525981
 0.18299494 0.30084732 0.95317339 0.07124013 0.42370739 0.74230851
 0.39327503 0.30608549 0.52755995 0.68451904 0.92358112 0.12069464
 0.57015554 0.54364173 0.3914261  0.9970799  0.86592304 0.4121005